In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Embedding, LSTM, Dense, Bidirectional, Dropout
from keras.callbacks import EarlyStopping, ReduceLROnPlateau
from sklearn.metrics import f1_score
from sklearn.utils.class_weight import compute_class_weight
from keras.optimizers import Adam

2023-08-31 19:15:28.978308: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
train = pd.read_excel('ClassificationDataset-train0.xlsx')
val = pd.read_excel('ClassificationDataset-valid0.xlsx')

In [3]:
max_words = 10000
max_len = 100
embedding_dim = 200  # Correct the embedding dimension to match GloVe embeddings
num_classes = len(np.unique(train['Label']))


In [4]:

le = LabelEncoder()
le.fit(train['Label'])

train_labels_encoded = le.transform(train['Label'])
val_labels_encoded = le.transform(val['Label'])
train_labels_onehot = np.eye(num_classes)[train_labels_encoded]
val_labels_onehot = np.eye(num_classes)[val_labels_encoded]


In [5]:
tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(train['Sentiment'])
train_sequences = tokenizer.texts_to_sequences(train['Sentiment'])
val_sequences = tokenizer.texts_to_sequences(val['Sentiment'])
train_data_pad = pad_sequences(train_sequences, maxlen=max_len)
val_data_pad = pad_sequences(val_sequences, maxlen=max_len)


In [6]:
embedding_index = {}
with open('glove.twitter.27B.200d.txt', encoding='utf-8') as f:
    for line in f:
        values = line.split()
        word = values[0]
        coef = np.asarray(values[1:], dtype='float32')
        embedding_index[word] = coef

embedding_matrix = np.zeros((max_words, embedding_dim))
for word, i in tokenizer.word_index.items():
    if i < max_words:
        embedding_vector = embedding_index.get(word)
        if embedding_vector is not None:
            embedding_matrix[i] = embedding_vector

In [7]:
model = Sequential()
model.add(Embedding(max_words, embedding_dim, input_length=max_len, weights=[embedding_matrix], trainable=False))
model.add(LSTM(128, return_sequences=True, dropout=0.2, recurrent_dropout=0.2)) 
#model.add(LSTM(64,return_sequences=True, dropout=0.2, recurrent_dropout=0.2))
model.add(LSTM(16,return_sequences=False, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(num_classes, activation='softmax'))

In [8]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 100, 200)          2000000   
                                                                 
 lstm (LSTM)                 (None, 100, 128)          168448    
                                                                 
 lstm_1 (LSTM)               (None, 16)                9280      
                                                                 
 dense (Dense)               (None, 3)                 51        
                                                                 
Total params: 2,177,779
Trainable params: 177,779
Non-trainable params: 2,000,000
_________________________________________________________________


In [9]:
adam_optimizer = Adam(learning_rate=0.001)
model.compile(optimizer=adam_optimizer, loss='categorical_crossentropy', metrics=['accuracy'])


In [10]:
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
#reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=2, min_lr=1e-6)


In [11]:
model.fit(train_data_pad, train_labels_onehot, epochs=50, batch_size=64,
          validation_data=(val_data_pad, val_labels_onehot),
           callbacks=[early_stopping])#class_weight=class_weights,


Epoch 1/50
44/44 [==============================] - 11s 158ms/step - loss: 0.7470 - accuracy: 0.6562 - val_loss: 0.7388 - val_accuracy: 0.6301
Epoch 2/50
44/44 [==============================] - 7s 149ms/step - loss: 0.6506 - accuracy: 0.7277 - val_loss: 0.6694 - val_accuracy: 0.7107
Epoch 3/50
44/44 [==============================] - 7s 149ms/step - loss: 0.5963 - accuracy: 0.7609 - val_loss: 0.5970 - val_accuracy: 0.7689
Epoch 4/50
44/44 [==============================] - 6s 147ms/step - loss: 0.5448 - accuracy: 0.7863 - val_loss: 0.5525 - val_accuracy: 0.7789
Epoch 5/50
44/44 [==============================] - 6s 147ms/step - loss: 0.5074 - accuracy: 0.7970 - val_loss: 0.5311 - val_accuracy: 0.7805
Epoch 6/50
44/44 [==============================] - 6s 148ms/step - loss: 0.4439 - accuracy: 0.8202 - val_loss: 0.5566 - val_accuracy: 0.7623
Epoch 7/50
44/44 [==============================] - 6s 146ms/step - loss: 0.4138 - accuracy: 0.8410 - val_loss: 0.5249 - val_accuracy: 0.7914
Epoch

In [12]:
val_predictions = model.predict(val_data_pad)
val_predictions_classes = np.argmax(val_predictions, axis=1)
f1_scores = f1_score(val_labels_encoded, val_predictions_classes, average='micro')
f1_scores

38/38 [==============================] - 1s 22ms/step


0.8162926018287614

In [14]:
model.save('dataset_0.h5')